# Non-covalent interactions benchmark

In [1]:
import pymolpro
import pandas as pd

In [2]:
backend = 'local'  # If preferred, change this to one of the backends in your ~/.sjef/molpro/backends.xml that is ssh-accessible
project_name = 'Non-covalent interactions benchmark'
parallel = None  # how many jobs to run at once

In [3]:
methods = {"HF": "df-hf", "MP2": "df-mp2", "LMP2": "df-lmp2", }
bases = ['aug-cc-pVDZ', 'aug-cc-pVTZ', 'aug-cc-pVQZ']

In [4]:
db = pymolpro.database.library("GMTKN55_S22").subset('small')

In [5]:
db.molecules.keys()

dict_keys(['02', '08', '01', '01a', '02a', '08a'])

In [6]:
results = {}
for method in methods:
    results[method] = {}
    for basis in bases:
        results[method][basis] = pymolpro.database.run(db, methods[method], basis, location=project_name,
                                                       backend=backend, parallel=parallel)

In [7]:
results['HF']['aug-cc-pVDZ']

In [8]:
db.molecules.keys()

dict_keys(['02', '08', '01', '01a', '02a', '08a'])

In [9]:
db.reactions.keys()

dict_keys(['2', '1', '8'])

In [10]:
extrapolations = {}
if 'aug-cc-pVDZ' in bases and 'aug-cc-pVTZ' in bases:
    extrapolations["aug-cc-pV[23]Z"] = {'x': 2, 'first': 'aug-cc-pVDZ', 'second': 'aug-cc-pVTZ'}
if 'aug-cc-pVTZ' in bases and 'aug-cc-pVQZ' in bases:
    extrapolations["aug-cc-pV[34]Z"] = {'x': 3, 'first': 'aug-cc-pVTZ', 'second': 'aug-cc-pVQZ'}
if 'aug-cc-pVQZ' in bases and 'aug-cc-pV5Z' in bases:
    extrapolations["aug-cc-pV[45]Z"] = {'x': 4, 'first': 'aug-cc-pVQZ', 'second': 'aug-cc-pV5Z'}

for extr in extrapolations:
    x = extrapolations[extr]['x']
    xbas = extrapolations[extr]['first']
    xpbas = extrapolations[extr]['second']
    for method in methods:
        results[method][extr] = pymolpro.database.basis_extrapolate(
            [results[method][xbas], results[method][xpbas]],
            [results['HF'][xbas], results['HF'][xpbas]],
            [x, x + 1]
        )
    bases.append(extr) if extr not in bases else ""


In [11]:
tokj = 2625.49963948
pd.set_option('display.precision', 2)
method_errors = pymolpro.database.compare([results[method]['aug-cc-pVQZ'] for method in methods], db)[
                    'reaction statistics'] * tokj
method_errors

KeyError: 'reaction statistics'

In [ ]:
tokj = 2625.49963948
pd.set_option('display.precision', 2)
method_errors = pymolpro.database.compare([results[method]['aug-cc-pVQZ'] for method in methods], db)[
                    'reaction energy errors'] * tokj
method_errors

In [ ]:
pd.set_option('display.precision', 2)
basis_errors = pymolpro.database.compare([results['LMP2'][basis] for basis in bases], db)[
                   'reaction statistics'] * tokj
basis_errors

In [ ]:
import matplotlib.pyplot as plt

methods_pruned = [method for method in methods if method != 'HF']
bases_pruned = ['aug-cc-pVTZ', 'aug-cc-pVQZ', 'aug-cc-pV[34]Z']
fig, panes = plt.subplots(nrows=1, ncols=len(bases_pruned), sharey=True, figsize=(18, 6))

for pane in range(len(bases_pruned)):
    data = []
    for method in methods_pruned:
        data.append(
            pymolpro.database.compare(results[method][bases_pruned[pane]],
                                      db)['reaction energy errors'].to_numpy()[:, 0] * tokj
        )
    panes[pane].violinplot(data, showmeans=True, showextrema=True, vert=True, bw_method='silverman')
    panes[pane].set_xticks(range(1, len(methods_pruned) + 1), labels=methods_pruned, rotation=-90)
    panes[pane].set_title(bases_pruned[pane])
panes[0].set_ylabel('Error / kJ/mol')
plt.savefig(project_name + ".violin.pdf")

In [ ]:
# with open(project_name + '.tex', 'w') as tf:
#     tf.write('\\def\\toprule{\\hline\\hline}\n\\def\\midrule{\\hline}\n\\def\\bottomrule{\\hline\\hline}')
#     tf.write(df_exp_reaction_meanerror.style.to_latex())
#     tf.write(df_exp_reaction_std.style.to_latex())
#     tf.write(df_exp_reaction_meanabserror.style.to_latex())
#     tf.write(df_exp_reaction_maxerror.style.to_latex())